In [62]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Prepare the SCARFLightning Module
from ts3l.pl_modules import SCARFLightning
from ts3l.utils.scarf_utils import SCARFDataset
from ts3l.utils import TS3LDataModule
from ts3l.utils.scarf_utils import SCARFConfig
from ts3l.utils.embedding_utils import IdentityEmbeddingConfig
from ts3l.utils.backbone_utils import MLPBackboneConfig
from pytorch_lightning import Trainer


# Evaluation
from sklearn.metrics import accuracy_score
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, SequentialSampler

In [63]:
TEST_DATA_PATH = "../data/test_data.csv"
TRAIN_DATA_PATH = "../data/train_data.csv"
UNLABELLED_DATA_PATH = "../data/unlabelled_data.csv"
PSEUDO_LABELLED_DATA_PATH = "../data/pseudo_labelled_scarf.csv"

In [64]:
test_df = pd.read_csv(TEST_DATA_PATH)
categorical_cols = ['Chemotherapy', 'Menopausal State', 'Radio Therapy',
                            'Hormone Therapy', 'Surgery-breast conserving',
                            'Surgery-mastectomy', 'Neoplasm Histologic Grade',
                            'Cellularity']
test_df[categorical_cols]

,Chemotherapy,Menopausal State,Radio Therapy,Hormone Therapy,Surgery-breast conserving,Surgery-mastectomy,Neoplasm Histologic Grade,Cellularity
0,0,1,1,1,0,1,3,0.5
1,0,1,1,1,0,1,2,0.5
2,1,1,1,0,0,1,3,1.0
3,0,1,1,1,0,1,3,1.0
4,1,1,1,0,0,1,2,1.0
...,...,...,...,...,...,...,...,...
112,0,1,0,1,0,1,3,0.5
113,1,0,0,1,0,1,2,0.5
114,0,1,0,1,0,1,3,1.0
115,0,1,1,1,1,0,1,0.5


In [65]:
def get_dataframes(test_path, train_path, unlabelled_path, with_clinical=False):
    # Load the data
    test_df = pd.read_csv(test_path)
    train_df = pd.read_csv(train_path)
    unlabelled_df = pd.read_csv(unlabelled_path)

    # Drop the columns that are not needed
    test_df = test_df.drop(columns=['DssTime', 'Event'])
    train_df = train_df.drop(columns=['DssTime', 'Event'])

    # Extract numerical and categorical columns
    # Numerical cols: Gene + Age
    numerical_cols = test_df.columns[:21].tolist()
    # But also Size
    numerical_cols.append('Size')
    # Categorical cols: Clinical
    categorical_cols = test_df.drop(columns=['Label', 'Size']).columns[21:].tolist()
    if not with_clinical:
        test_df = test_df.drop(columns=categorical_cols)
        train_df = train_df.drop(columns=categorical_cols)
        unlabelled_df = unlabelled_df.drop(columns=categorical_cols)
        categorical_cols = []
    else:
        categorical_cols = ['Chemotherapy', 'Menopausal State', 'Radio Therapy',
                            'Hormone Therapy', 'Surgery-breast conserving',
                            'Surgery-mastectomy', 'Neoplasm Histologic Grade',
                            'Cellularity']
        not_cols=[]
        # The model has problems with these columns
        test_df = test_df.drop(columns=not_cols)
        train_df = train_df.drop(columns=not_cols)
        unlabelled_df = unlabelled_df.drop(columns=not_cols)

    print(f'Train data shape: {train_df.shape}')
    print(f'Test data shape: {test_df.shape}')
    print(f'Unlabelled data shape: {unlabelled_df.shape}')
    print(f'Numerical columns: {numerical_cols}')
    if with_clinical:
        print(f'Categorical columns: {categorical_cols}')
    return test_df, train_df, unlabelled_df, numerical_cols, categorical_cols

In [66]:
test_data, train_data, unlabelled_data, numerical_cols, categorical_cols = get_dataframes(
    TEST_DATA_PATH,
    TRAIN_DATA_PATH,
    UNLABELLED_DATA_PATH,
    with_clinical=True)

Train data shape: (465, 31)
Test data shape: (117, 31)
Unlabelled data shape: (1168, 30)
Numerical columns: ['ESR1', 'PGR', 'ERBB2', 'MKI67', 'PLAU', 'ELAVL1', 'EGFR', 'BTRC', 'FBXO6', 'SHMT2', 'KRAS', 'SRPK2', 'YWHAQ', 'PDHA1', 'EWSR1', 'ZDHHC17', 'ENO1', 'DBN1', 'PLK1', 'GSK3B', 'Age', 'Size']
Categorical columns: ['Chemotherapy', 'Menopausal State', 'Radio Therapy', 'Hormone Therapy', 'Surgery-breast conserving', 'Surgery-mastectomy', 'Neoplasm Histologic Grade', 'Cellularity']


In [67]:
test_data.head()

,ESR1,PGR,ERBB2,MKI67,PLAU,ELAVL1,EGFR,BTRC,FBXO6,SHMT2,...,Menopausal State,Size,Radio Therapy,Chemotherapy,Hormone Therapy,Neoplasm Histologic Grade,Cellularity,Surgery-breast conserving,Surgery-mastectomy,Label
0,11.239750,5.954311,9.739996,6.046045,10.040187,5.905724,5.881255,6.538235,7.260572,10.774752,...,1,31,1,0,1,3,0.5,0,1,1
1,10.927313,7.002502,10.033753,5.568993,8.306619,6.547491,5.733367,6.128118,7.917904,9.514045,...,1,22,1,0,1,2,0.5,0,1,0
2,6.312633,5.305683,9.068778,5.919384,8.210977,5.896152,5.634379,5.625037,7.684047,11.422518,...,1,40,1,1,0,3,1.0,0,1,1
3,9.185200,5.480888,9.580607,5.655789,7.756504,6.026981,6.008594,6.269051,7.428641,9.478211,...,1,150,1,0,1,3,1.0,0,1,1
4,7.249462,5.164281,10.233184,5.721403,8.918334,6.392132,5.588450,6.062906,7.968933,9.578638,...,1,45,1,1,0,2,1.0,0,1,1


In [68]:
full_X_train = train_data.drop(columns=['Label'])
full_y_train = train_data['Label']

X_test = test_data.drop(columns=['Label'])
y_test = test_data['Label']

In [69]:
# Split the train_data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    full_X_train,
    full_y_train,
    test_size=0.2,
    random_state=42,
    stratify=full_y_train)

print(f'Training data shape: {X_train.shape}')
print(f'Validation data shape: {X_val.shape}')

Training data shape: (372, 30)
Validation data shape: (93, 30)


In [70]:
metric = "accuracy_score"
input_dim = X_train.shape[1]
pretraining_head_dim = 1024
output_dim = 2
head_depth = 2
dropout_rate = 0.04

corruption_rate = 0.6

batch_size = 128
max_epochs = 10

In [71]:
embedding_config = IdentityEmbeddingConfig(input_dim = input_dim)
backbone_config = MLPBackboneConfig(input_dim = embedding_config.output_dim)

In [72]:
config = SCARFConfig( 
    task="classification",
    loss_fn="CrossEntropyLoss",
    metric=metric, metric_hparams={},
    embedding_config=embedding_config,
    backbone_config=backbone_config,
    pretraining_head_dim=pretraining_head_dim,
    output_dim=output_dim,
    head_depth=head_depth,
    dropout_rate=dropout_rate,
    corruption_rate = corruption_rate
)

In [73]:
pl_scarf = SCARFLightning(config)

Seed set to 42


In [74]:
### First Phase Learning
train_ds = SCARFDataset(
    X_train,
    unlabeled_data=unlabelled_data,
    config = config,
    continuous_cols=numerical_cols,
    category_cols=categorical_cols)

valid_ds = SCARFDataset(
    X_val,
    config=config,
    continuous_cols=numerical_cols,
    category_cols=categorical_cols
)

datamodule = TS3LDataModule(train_ds, valid_ds, batch_size=batch_size, train_sampler="random")


In [75]:
trainer = Trainer(
    accelerator = 'cpu',
    max_epochs = max_epochs,
    num_sanity_val_steps = 2,
    )

trainer.fit(pl_scarf, datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/sonk/envs/pandas/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(

  | Name             | Type             | Params | Mode 
--------------------------------------------------------------
0 | task_loss_fn     | CrossEntropyLoss | 0      | train
1 | contrastive_loss | NTXentLoss       | 0      | train
2 | model            | SCARF            | 1.2 M  | train
--------------------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M  

/home/sonk/envs/pandas/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (13) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 9: 100%|██████████| 13/13 [00:17<00:00,  0.75it/s, v_num=69, train_loss=4.940, val_loss=5.180]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 13/13 [00:17<00:00,  0.74it/s, v_num=69, train_loss=4.940, val_loss=5.180]


In [76]:
### Second Phase Learning

pl_scarf.set_second_phase()


In [77]:
train_ds = SCARFDataset(
    X_train,
    y_train.values,
    continuous_cols=numerical_cols,
    category_cols=categorical_cols,
    is_second_phase=True)

valid_ds = SCARFDataset(
    X_val,
    y_val.values,
    continuous_cols=numerical_cols,
    category_cols=categorical_cols,
    is_second_phase=True)

datamodule = TS3LDataModule(train_ds, valid_ds, batch_size = batch_size, train_sampler="weighted")


In [78]:
trainer = Trainer(
                    accelerator = 'cpu',
                    max_epochs = max_epochs,
                    num_sanity_val_steps = 2,
    )

trainer.fit(pl_scarf, datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/sonk/envs/pandas/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(

  | Name             | Type             | Params | Mode 
--------------------------------------------------------------
0 | task_loss_fn     | CrossEntropyLoss | 0      | train
1 | contrastive_loss | NTXentLoss       | 0      | train
2 | model            | SCARF            | 1.2 M  | train
--------------------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M  

/home/sonk/envs/pandas/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/home/sonk/envs/pandas/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 1:   0%|          | 0/3 [00:00<?, ?it/s, v_num=70, train_loss=0.912, train_accuracy_score=0.524, val_accuracy_score=0.602, val_loss=0.678]        

/home/sonk/envs/pandas/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 2:   0%|          | 0/3 [00:00<?, ?it/s, v_num=70, train_loss=0.793, train_accuracy_score=0.581, val_accuracy_score=0.613, val_loss=0.662]        

...

Epoch 9:   0%|          | 0/3 [00:00<?, ?it/s, v_num=70, train_loss=0.853, train_accuracy_score=0.559, val_accuracy_score=0.624, val_loss=0.635]        

/home/sonk/envs/pandas/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 9: 100%|██████████| 3/3 [00:01<00:00,  1.96it/s, v_num=70, train_loss=0.846, train_accuracy_score=0.546, val_accuracy_score=0.634, val_loss=0.631]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 3/3 [00:01<00:00,  1.89it/s, v_num=70, train_loss=0.846, train_accuracy_score=0.546, val_accuracy_score=0.634, val_loss=0.631]


In [79]:
test_ds = SCARFDataset(
    X_test,
    continuous_cols=numerical_cols,
    category_cols=categorical_cols,
    is_second_phase=True)

test_dl = DataLoader(
    test_ds,
    batch_size,
    shuffle=False,
    sampler=SequentialSampler(test_ds),
    num_workers=4)


In [80]:
preds = trainer.predict(pl_scarf, test_dl)
        
preds = F.softmax(torch.concat([out.cpu() for out in preds]).squeeze(),dim=1)

accuracy = accuracy_score(y_test, preds.argmax(1))

print("Accuracy %.2f" % accuracy)

/home/sonk/envs/pandas/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 337.71it/s]
Accuracy 0.72
